see whatsapp Harris post 15/04/2025 @19:57 #full scrape with no headless

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import random
import threading
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime 

# 設置 Chrome 選項（移除 Headless 模式）
def setup_chrome_options():
    options = Options()
    options.add_argument('--lang=en')  # 設置語言為英文
    # 移除 --headless，改用正常模式
    return options

# 鞋碼轉換對照表，包含 UK5 至 UK10
size_conversion = {
    "UK5": {"US": "6.5", "EU": "38"},
    "UK5.5": {"US": "7", "EU": "38.7"},
    "UK6": {"US": "7.5", "EU": "39.3"},
    "UK6.5": {"US": "8", "EU": "40"},
    "UK7": {"US": "8.5", "EU": "40.7"},
    "UK7.5": {"US": "9", "EU": "41.3"},
    "UK8": {"US": "9.5", "EU": "42"},
    "UK8.5": {"US": "10", "EU": "42.7"},
    "UK9": {"US": "10.5", "EU": "43.3"},
    "UK9.5": {"US": "11", "EU": "44"},
    "UK10": {"US": "11.5", "EU": "44.7"}
}

# US 轉 UK 的反向對照表
us_to_uk = {v["US"]: k.replace("UK", "") for k, v in size_conversion.items()}

# 定義鞋碼範圍，包含 UK5 至 UK10
uk_sizes = ["5", "5.5", "6", "6.5", "7", "7.5", "8", "8.5", "9", "9.5", "10"]

# 定義鞋碼欄位名稱映射
size_column_names = {
    "5": "uk_size_5",
    "5.5": "uk_size_5_5",
    "6": "uk_size_6",
    "6.5": "uk_size_6_5",
    "7": "uk_size_7",
    "7.5": "uk_size_7_5",
    "8": "uk_size_8",
    "8.5": "uk_size_8_5",
    "9": "uk_size_9",
    "9.5": "uk_size_9_5",
    "10": "uk_size_10"
}

# 定義折扣加權值映射（統一為小寫）
discount_weights = {
    "vip 10% off": 90,
    "vip 15% off": 85,
    "10% off": 90,
    "buy 1 get 10% off": 90,
    "buy 2 get 20% off": 80,
    "buy 1 get 15% off": 85,
    "buy 3 get 30% off": 70,
    "buy 2 get 10% off": 90,
    "no discount": 100
}

# 計算折扣加權值的函數（忽略大小寫）
def calculate_discount_weight(discount_text):
    discount_text_lower = discount_text.lower()
    if discount_text_lower in discount_weights:
        return discount_weights[discount_text_lower]
    return 1.0

# 鎖，用於保護共享資源
lock = threading.Lock()

# 每個執行緒的抓取函數（抓取內頁資料）
def fetch_details(index, product_name, href, size_status_list, sku_list, global_index):
    driver = webdriver.Chrome(options=setup_chrome_options())
    try:
        print(f'執行緒 {threading.current_thread().name} 正在處理第 {global_index+1} 個產品: {product_name}')
        driver.get(href)
        print(f'執行緒 {threading.current_thread().name} 已進入產品 {global_index+1} 的詳情頁面: {product_name}')
        time.sleep(random.uniform(3, 5))

        try:
            popup_close = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Close") or contains(text(), "OK") or @aria-label="Close"]'))
            )
            driver.execute_script("arguments[0].click();", popup_close)
            print(f'執行緒 {threading.current_thread().name} 已關閉詳情頁面彈窗')
        except:
            print(f'執行緒 {threading.current_thread().name} 無詳情頁面彈窗或無法關閉')

        # 抓取鞋碼並轉換為數字
        size_status = {size_column_names[uk_size]: 0 for uk_size in uk_sizes}
        try:
            size_elements = WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "swatch-option text") and (contains(@data-option-label, "UK") or contains(@data-option-label, "US"))]'))
            )
            if size_elements:
                for size in size_elements:
                    size_label = size.get_attribute("data-option-label")
                    if not size_label:
                        continue
                    uk_size = None
                    if "UK" in size_label:
                        uk_size = size_label.replace("UK", "").strip()
                    elif "US" in size_label:
                        us_size = size_label.replace("US", "").strip()
                        uk_size = us_to_uk.get(us_size)
                    if uk_size and uk_size in uk_sizes:
                        style = size.get_attribute("style")
                        status = 2 if style == "" else 1  # 有貨: 2, 缺貨: 1
                        size_status[size_column_names[uk_size]] = status
                        print(f'執行緒 {threading.current_thread().name} 鞋碼 UK{uk_size}: {status}')
            else:
                print(f'執行緒 {threading.current_thread().name} 產品 {global_index+1} 無鞋碼')
        except:
            print(f'執行緒 {threading.current_thread().name} 產品 {global_index+1} 抓取失敗：鞋碼元素未找到')
            size_status = {size_column_names[uk_size]: 0 for uk_size in uk_sizes}

        # 抓取 SKU
        try:
            sku_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="value" and @itemprop="sku"]'))
            )
            sku = sku_element.text.strip()
            print(f'執行緒 {threading.current_thread().name} 產品 {global_index+1} SKU: {sku}')
        except:
            sku = "無 SKU"
            print(f'執行緒 {threading.current_thread().name} 產品 {global_index+1} 無 SKU')

        with lock:
            size_status_list[index] = size_status
            sku_list[index] = sku

    except Exception as e:
        print(f'執行緒 {threading.current_thread().name} 產品 {global_index+1} 抓取失敗: {e}')
        with lock:
            size_status_list[index] = {size_column_names[uk_size]: 0 for uk_size in uk_sizes}
            sku_list[index] = "抓取失敗"
    finally:
        driver.quit()

# 主程式
options = setup_chrome_options()
driver = webdriver.Chrome(options=options)

# 設置爬取的頁數範圍
start_page = 1  # 起始頁數
end_page = 2   # 結束頁數
base_url = 'https://catalog.hkstore.com/catalog_en_us/women/footwear'  # 基礎網址

# 初始化清單儲存所有頁面的資料
all_brand_list = []
all_product_name_list = []
all_gender_list = []
all_type_list = []
all_price_list = []
all_discount1_list = []
all_discount2_list = []
all_discount3_list = []
all_discount1_weighted_list = []
all_discount2_weighted_list = []
all_discount3_weighted_list = []
all_href_list = []
all_size_status_list = []
all_sku_list = []

# 用於記錄全局產品序號（用於 scrape_id 和日誌）
global_product_index = 0

# 分頁處理，指定範圍
current_page = start_page
while current_page <= end_page:
    # 構造目標頁面的網址
    target_url = f"{base_url}?p={current_page}"
    print(f"\n開始處理第 {current_page} 頁，目標 URL: {target_url}\n")
    
    # 直接訪問目標頁面
    try:
        driver.get(target_url)
        time.sleep(5)  # 等待頁面載入
    except Exception as e:
        print(f"無法訪問第 {current_page} 頁，錯誤: {e}")
        break

    # 檢查當前 URL，確認頁面
    current_url = driver.current_url
    print(f"當前 URL: {current_url}")
    if str(current_page) not in current_url:
        print(f"警告：當前 URL 不包含預期的頁數 {current_page}，可能未正確進入頁面")
        break

    # 檢查是否有彈窗並關閉
    try:
        cookie_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Accept") or contains(text(), "Agree") or contains(text(), "OK")]'))
        )
        driver.execute_script("arguments[0].click();", cookie_button)
        print("已關閉彈窗")
    except:
        print("無彈窗或無法關閉")

    # 初始化當前頁面的資料清單
    brand_list = []
    product_name_list = []
    gender_list = []
    type_list = []
    price_list = []
    discount1_list = []
    discount2_list = []
    discount3_list = []
    discount1_weighted_list = []
    discount2_weighted_list = []
    discount3_weighted_list = []
    href_list = []
    
    # 滾動頁面以載入所有產品
    previous_count = 0
    max_attempts = 5
    attempt = 0
    while attempt < max_attempts:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        result_list = driver.find_elements(By.XPATH, '//div[@class="product-item-info"]')
        current_count = len(result_list)
        print(f'第 {current_page} 頁滾動後找到 {current_count} 個產品')
        if current_count == previous_count:
            break
        previous_count = current_count
        attempt += 1

    # 檢查是否有產品，如果沒有則跳出
    if current_count == 0:
        print(f"第 {current_page} 頁沒有找到產品，停止爬取")
        break

    # 抓取當前頁面的品牌
    brand_elements = driver.find_elements(By.XPATH, '//div[@class="product-item-brand"]')
    print(f'第 {current_page} 頁找到 {len(brand_elements)} 個品牌')
    for brand in brand_elements:
        print(brand.text)
        brand_list.append(brand.text)

    # 抓取當前頁面的產品名稱
    product_name_elements = driver.find_elements(By.XPATH, '//strong[@class="product name product-item-name"]')
    print(f'第 {current_page} 頁找到 {len(product_name_elements)} 個產品名稱')
    for name in product_name_elements:
        print(name.text)
        product_name_list.append(name.text)

    # 抓取當前頁面的 Gender 和 Type
    gender_type_elements = driver.find_elements(By.XPATH, '//div[@class="product-item-gender-type"]')
    print(f'第 {current_page} 頁找到 {len(gender_type_elements)} 個性別類型')
    for gender_type in gender_type_elements:
        text = gender_type.text.strip()
        print(f'原始 GenderType: {text}')
        if text:
            parts = text.split(" ")
            gender = parts[0]
            type_ = " ".join(parts[1:]) if len(parts) > 1 else "未知類型"
        else:
            gender = "未知性別"
            type_ = "未知類型"
        print(f'Gender: {gender}, Type: {type_}')
        gender_list.append(gender)
        type_list.append(type_)

    # 抓取當前頁面的最終價格
    price_elements = driver.find_elements(By.XPATH, '//span[@data-price-type="finalPrice"]//span[@class="price"]')
    print(f'第 {current_page} 頁找到 {len(price_elements)} 個最終價格')
    for price in price_elements:
        print(price.text)
        price_text = price.text.replace("HK$", "").replace(",", "")
        try:
            price_value = float(price_text)
        except ValueError:
            price_value = 0.0
        price_list.append(price_value)

    # 抓取當前頁面的折扣並拆分
    for item in result_list:
        left_discount = item.find_elements(By.XPATH, './/div[contains(@class, "cataloglabel-bottom-left")]//span[contains(text(), "Off") or contains(text(), "Buy")]')
        right_discount = item.find_elements(By.XPATH, './/div[contains(@class, "cataloglabel-botton-right")]//span[contains(text(), "Off") or contains(text(), "Buy")]')
        
        # 抓取左折扣
        left_text = left_discount[0].text.strip() if left_discount else ""
        left_discounts = []
        if left_text and left_text not in ["無左折扣", ""]:
            left_discounts = [d.strip() for d in left_text.split("|")]
        
        # 抓取右折扣
        right_text = right_discount[0].text.strip() if right_discount else ""
        right_discounts = []
        if right_text and right_text not in ["無右折扣", ""]:
            right_discounts = [d.strip() for d in right_text.split("|")]
        
        # 合併所有有效的折扣
        all_discounts = left_discounts + right_discounts
        if not all_discounts:
            all_discounts = ["No Discount"]
        all_discounts = all_discounts[:3]
        while len(all_discounts) < 3:
            all_discounts.append("No Discount")
        
        discount1_list.append(all_discounts[0])
        discount2_list.append(all_discounts[1])
        discount3_list.append(all_discounts[2])
        
        discount1_weighted_list.append(calculate_discount_weight(all_discounts[0]))
        discount2_weighted_list.append(calculate_discount_weight(all_discounts[1]))
        discount3_weighted_list.append(calculate_discount_weight(all_discounts[2]))
        
        print(f'折扣1: {all_discounts[0]} (加權值: {discount1_weighted_list[-1]}) | 折扣2: {all_discounts[1]} (加權值: {discount2_weighted_list[-1]}) | 折扣3: {all_discounts[2]} (加權值: {discount3_weighted_list[-1]})')

    # 抓取當前頁面的 href
    for item in result_list:
        try:
            link = item.find_element(By.XPATH, './/a[.//img[@class="product-image-photo"]]')
            href = link.get_attribute("href")
            href_list.append(href)
        except:
            href_list.append("無連結")

    # 初始化當前頁面的鞋碼狀態和 SKU 清單
    total_products_in_page = len(product_name_list)
    print(f"當前頁面共有 {total_products_in_page} 個產品")
    size_status_list = [None] * total_products_in_page
    sku_list = [None] * total_products_in_page

    # 使用多執行緒抓取當前頁面每雙鞋的內頁資料
    max_workers = 4
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for index in range(total_products_in_page):
            if href_list[index] == "無連結":
                print(f'第 {current_page} 頁產品 {index+1} 無連結，跳過')
                size_status_list[index] = {size_column_names[uk_size]: 0 for uk_size in uk_sizes}
                sku_list[index] = "無連結"
                continue
            future = executor.submit(fetch_details, index, product_name_list[index], href_list[index], size_status_list, sku_list, global_product_index)
            futures.append(future)
            global_product_index += 1  # 更新全局產品序號

        for future in futures:
            future.result()

    # 將當前頁面的資料追加到總清單中
    all_brand_list.extend(brand_list)
    all_product_name_list.extend(product_name_list)
    all_gender_list.extend(gender_list)
    all_type_list.extend(type_list)
    all_price_list.extend(price_list)
    all_discount1_list.extend(discount1_list)
    all_discount2_list.extend(discount2_list)
    all_discount3_list.extend(discount3_list)
    all_discount1_weighted_list.extend(discount1_weighted_list)
    all_discount2_weighted_list.extend(discount2_weighted_list)
    all_discount3_weighted_list.extend(discount3_weighted_list)
    all_href_list.extend(href_list)
    all_size_status_list.extend(size_status_list)
    all_sku_list.extend(sku_list)

    print(f"第 {current_page} 頁資料已全部抓取，總共處理了 {global_product_index} 個產品")

    # 更新頁數
    current_page += 1
    print(f"頁數已更新為 {current_page}")

# 關閉主頁面的瀏覽器
driver.quit()

# 調整清單長度（確保所有欄位行數一致）
max_length = max(len(all_brand_list), len(all_product_name_list), len(all_gender_list), len(all_type_list), len(all_price_list), len(all_discount1_list), len(all_discount2_list), len(all_discount3_list), len(all_sku_list))
for lst in [all_brand_list, all_product_name_list, all_gender_list, all_type_list, all_price_list, all_discount1_list, all_discount2_list, all_discount3_list, all_discount1_weighted_list, all_discount2_weighted_list, all_discount3_weighted_list, all_sku_list, all_size_status_list]:
    while len(lst) < max_length:
        lst.append("N/A")

# 根據 url 判斷 website 的值
if base_url.startswith('https://marathonsports.hkstore.com/marathon_en_us/'):
    website_value = 0
    website_code = "01"  # 馬拉松
elif base_url.startswith('https://catalog.hkstore.com/catalog_en_us/'):
    website_value = 1
    website_code = "02"  # Catalog
else:
    website_value = 2
    website_code = "00"  # 其他網站

# 生成 scrape_id（格式：網站 + 日期 + 編號）
scrape_date = datetime.now().strftime("%y%m%d")  # 日期格式為 YYMMDD，例如 250414
scrape_ids = [f"{website_code}{scrape_date}{str(i).zfill(2)}" for i in range(len(all_product_name_list))]

# 整理成表格
data = {
    'scrape_id': scrape_ids,
    'shoe_brand': all_brand_list,
    'shoe_name': all_product_name_list,
    'shoe_gender': all_gender_list,
    'shoe_type': all_type_list,
    'price': all_price_list,
    'discount_1_text': all_discount1_list,
    'discount_1_weighted': all_discount1_weighted_list,
    'discount_2_text': all_discount2_list,
    'discount_2_weighted': all_discount2_weighted_list,
    'discount_3_text': all_discount3_list,
    'discount_3_weighted': all_discount3_weighted_list,
    'shoe_sku': all_sku_list,
    'website': [website_value] * len(all_product_name_list),
    'page_ranking': list(range(len(all_product_name_list)))
}

# 將鞋碼狀態加入表格
for uk_size in uk_sizes:
    column_name = size_column_names[uk_size]
    data[column_name] = [status[column_name] for status in all_size_status_list]

df = pd.DataFrame(data)

# 添加抓取日期
scrape_date_full = datetime.now().strftime("%Y-%m-%d")
df['scrape_time'] = scrape_date_full

# 分成兩個表格並輸出為 CSV
date_prefix = datetime.now().strftime("%Y%m%d")  # 格式為 YYYYMMDD，例如 20250414

# 第一個表格：catalog_shoes.csv
df_shoes = df[['shoe_sku', 'shoe_name', 'shoe_brand', 'shoe_type', 'shoe_gender']]
df_shoes.to_csv(f'{date_prefix}_catalog_shoes.csv', index=False, encoding='utf-8-sig')
print(f"已將第一個表格儲存為 '{date_prefix}_catalog_shoes.csv'")

# 第二個表格：catalog_scrape_data.csv
df_scrape_data = df[['scrape_id', 'shoe_sku', 'price', 
                     'discount_1_text', 'discount_1_weighted', 
                     'discount_2_text', 'discount_2_weighted', 
                     'discount_3_text', 'discount_3_weighted',
                     'uk_size_5', 'uk_size_5_5', 'uk_size_6', 'uk_size_6_5', 'uk_size_7',
                     'uk_size_7_5', 'uk_size_8', 'uk_size_8_5', 'uk_size_9', 'uk_size_9_5', 'uk_size_10',
                     'page_ranking', 'website', 'scrape_time']]
df_scrape_data.to_csv(f'{date_prefix}_catalog_scrape_data.csv', index=False, encoding='utf-8-sig')
print(f"已將第二個表格儲存為 '{date_prefix}_catalog_scrape_data.csv'")

# 顯示兩個表格
print("\n第一個表格 (catalog_shoes):")
print(df_shoes)
print("\n第二個表格 (catalog_scrape_data):")
print(df_scrape_data)


開始處理第 1 頁，目標 URL: https://catalog.hkstore.com/catalog_en_us/women/footwear?p=1

當前 URL: https://catalog.hkstore.com/catalog_en_us/women/footwear?p=1
無彈窗或無法關閉
第 1 頁滾動後找到 24 個產品
第 1 頁滾動後找到 24 個產品
第 1 頁找到 24 個品牌
Hoka
Adidas
Adidas
Hoka
Hoka
Converse
Converse
Converse
Converse
Converse
Converse
Converse
Converse
Converse
Adidas
Adidas
New Balance
New Balance
New Balance
New Balance
New Balance
New Balance
New Balance
New Balance
第 1 頁找到 24 個產品名稱
Clifton 10
SL 72 RS
SL 72 OG
Clifton 10
Clifton 10
Run Star Trainer
Omni Trainer
Converse Omega Trainer
Converse Omega Trainer
Run Star Trainer
Omni Trainer
Converse Omega Trainer
Run Star Trainer
Run Star Trainer
Samba OG
Samba OG
530
530
327
1906R
1906R
327
327
530
第 1 頁找到 24 個性別類型
原始 GenderType: Women Running Shoes
Gender: Women, Type: Running Shoes
原始 GenderType: Unisex Sneakers
Gender: Unisex, Type: Sneakers
原始 GenderType: Women Sneakers
Gender: Women, Type: Sneakers
原始 GenderType: Women Running Shoes
Gender: Women, Type: Running Shoes
原始 Gen